# Crypto Technical Screening

Here, we'll do some technical screening and explore ways to combine various technical indicators in our crypto universe

In [10]:
import requests
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.technical_indicators import *

# Use your absolute path to our top 500 coins csv
top_coins_path = '/Users/markwindsor/Desktop/atlas_trade/src/data/top_coins.csv'

cryptoverse_df = pd.read_csv(top_coins_path)

# We'll load some price data into our dictionary 
cryptoverse_prices = {}

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    # Retrieves OHLCV and mcap data 
    url = f"https://91j3dag4m9.execute-api.us-east-1.amazonaws.com/master/getCMCPriceHistory/{id}?startDate=2021-01-01"
    response = requests.get(url)
    data = response.json()
    table = []
    # Response gives us o,h,l,c,v,mcap keys which we need to change our column names like open, close etc...
    # because thats what our utility functions accept such as our technical indicator functions
    for entry in data["priceHistory"]:
        table.append([
            entry["o"],
            entry["h"],
            entry["l"],
            entry["c"],
            entry["v"],
            entry["mcap"]
        ])

    # Create a DataFrame from the extracted data
    columns = ["open", "high", "low", "close", "volume", "marketCap"]
    prices_df = pd.DataFrame(table, columns=columns)
    prices_df["log_returns"] = np.log(prices_df.close / prices_df.close.shift(1))
    prices_df["cumulative_log_returns"] = prices_df["log_returns"].cumsum()
    cryptoverse_prices[id] = prices_df


/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Use

## RSI

Lets start with the RSI. Here, we'll explore screening the top cryptos with regards to the RSI. 0-30 = oversold, 30-70 = Neutral, 70-100 = Overbought

In [11]:
oversold = []
overbought = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    rsi_df = relative_strength_index(cryptoverse_prices[id])
    last_row_rsi = rsi_df.iloc[-1]
    if (last_row_rsi['rsi'] < 30 and last_row_rsi['rsi'] > 0):
        oversold.append(f"id: {id}, symbol: {symbol}, name: {name}")
    if (last_row_rsi['rsi'] < 100 and last_row_rsi['rsi'] > 70):
        overbought.append(f"id: {id}, symbol: {symbol}, name: {name}")

print('RSI Below 30 (Oversold): ', oversold)
print('RSI 70 - 100 (Overbought): ', overbought)


RSI Below 30 (Oversold):  ['id: 4642, symbol: HBAR, name: HEDERA', 'id: 11840, symbol: OP, name: OPTIMISM', 'id: 2087, symbol: KCS, name: KUCOIN TOKEN', 'id: 7080, symbol: GALA, name: GALA', 'id: 5692, symbol: COMP, name: COMPOUND', 'id: 4041, symbol: MX, name: MX TOKEN', 'id: 9258, symbol: XCH, name: CHIA', 'id: 11865, symbol: BONE, name: BONE SHIBASWAP', 'id: 12220, symbol: OSMO, name: OSMOSIS', 'id: 8677, symbol: XYM, name: SYMBOL', 'id: 14783, symbol: MAGIC, name: MAGIC', 'id: 23254, symbol: CORE, name: CORE DAO', 'id: 5190, symbol: FLEX, name: FLEX', 'id: 11079, symbol: BRISE, name: BITGERT', 'id: 2132, symbol: POWR, name: POWERLEDGER', 'id: 17591, symbol: NYM, name: NYM', 'id: 22265, symbol: OAS, name: OASYS', 'id: 1762, symbol: ERG, name: ERGO', 'id: 9674, symbol: WILD, name: WILDER WORLD', 'id: 20886, symbol: ASTRAFER, name: ASTRAFER', 'id: 21178, symbol: MRS, name: METARS GENESIS', 'id: 3600, symbol: HUM, name: HUMANSCAPE', 'id: 12147, symbol: SYN, name: SYNAPSE', 'id: 4189, s

## MACD

Lets explore the macd

In [12]:
macd_above_signal = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    macd_df = macd(cryptoverse_prices[id])
    last_row_macd = macd_df.iloc[-1]
    if (last_row_macd['macd'] > last_row_macd['signal']):
        macd_above_signal.append(f"id: {id}, symbol: {symbol}, name: {name}")

print("MACD Line greater than signal line: ", macd_above_signal)

KeyError: 1